### Librerías

In [4]:
import zipfile
import sqlite3
import requests
import pandas as pd
from io import BytesIO

### Descarga de datos

In [12]:
# Link de descarga de los datos
url = "https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/etv/historicos/2023/datos_abiertos_dengue_261023.zip"

# Descarga los datos y los guarda en un objeto de tipo BytesIO
response = requests.get(url)
zip_file = BytesIO(response.content)

# Extrae los datos del objeto zip_file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('.\\data') # Cambia 'ruta' por la ruta donde deseas extraer los archivos

In [13]:
# Leer el archivo de datos 
df = pd.read_csv('data/dengue_abierto.csv')

# Muestra los primeros registros para verificar que la lectura fue exitosa
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2023-10-23,1004648,1,17,23,5,2,2,23,5,...,2,2,2,2,5.0,1,3,2,23,5
1,2023-10-23,1004661,2,25,23,5,2,2,23,5,...,2,2,2,2,5.0,1,5,3,23,5
2,2023-10-23,1004693,2,8,19,21,2,2,19,21,...,2,2,2,2,5.0,1,5,3,19,21
3,2023-10-23,1004728,1,64,25,1,2,2,25,1,...,2,2,2,2,5.0,1,5,3,25,1
4,2023-10-23,1004754,1,30,6,2,2,2,6,2,...,2,2,2,2,5.0,1,5,3,6,2


In [7]:
# Crear diccionarios csv
diccionario = pd.read_excel('Catálogos_Dengue.xlsx', sheet_name=None)

# Sacamos los diccionarios en formato csv leyendo cada sheet del archivo
for sheet_name, sheet_data in diccionario.items():
    # Reemplaza los espacios en el nombre de la hoja con guiones bajos
    nombre_archivo_csv = f"{sheet_name.replace(' ', '_')}.csv"
    sheet_data.to_csv(f'data/{nombre_archivo_csv}', index=False)

In [8]:
# Crear la base de datos SQLite
conn = sqlite3.connect('mcd.db')

# Enviar el DataFrame a la base de datos SQLite
df.to_sql('data/dengue_2023', conn, index=False, if_exists='replace')

# Cerrar la conexión
conn.close()

### Limpieza de datos

In [9]:
# Leer el dataframe desde la base de datos SQLite
conn = sqlite3.connect('mcd.db')
df = pd.read_sql('SELECT * FROM dengue_2023', conn)

# Muestra los primeros registros para verificar que la lectura fue exitosa
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2023-10-23,1004648,1,17,23,5,2,2,23,5,...,2,2,2,2,5.0,1,3,2,23,5
1,2023-10-23,1004661,2,25,23,5,2,2,23,5,...,2,2,2,2,5.0,1,5,3,23,5
2,2023-10-23,1004693,2,8,19,21,2,2,19,21,...,2,2,2,2,5.0,1,5,3,19,21
3,2023-10-23,1004728,1,64,25,1,2,2,25,1,...,2,2,2,2,5.0,1,5,3,25,1
4,2023-10-23,1004754,1,30,6,2,2,2,6,2,...,2,2,2,2,5.0,1,5,3,6,2


In [10]:
# Verificamos el tipo de dato de cada columna
df.dtypes

FECHA_ACTUALIZACION        object
ID_REGISTRO                 int64
SEXO                        int64
EDAD_ANOS                   int64
ENTIDAD_RES                 int64
MUNICIPIO_RES               int64
HABLA_LENGUA_INDIG          int64
INDIGENA                    int64
ENTIDAD_UM_NOTIF            int64
MUNICIPIO_UM_NOTIF          int64
INSTITUCION_UM_NOTIF        int64
FECHA_SIGN_SINTOMAS        object
TIPO_PACIENTE               int64
HEMORRAGICOS                int64
DIABETES                    int64
HIPERTENSION                int64
ENFERMEDAD_ULC_PEPTICA      int64
ENFERMEDAD_RENAL            int64
INMUNOSUPR                  int64
CIRROSIS_HEPATICA           int64
EMBARAZO                    int64
DEFUNCION                   int64
DICTAMEN                  float64
TOMA_MUESTRA                int64
RESULTADO_PCR               int64
ESTATUS_CASO                int64
ENTIDAD_ASIG                int64
MUNICIPIO_ASIG              int64
dtype: object

In [11]:
# Cambiamos el tipo de dato de las columnas a las que les corresponde
df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_SIGN_SINTOMAS'] = pd.to_datetime(df['FECHA_SIGN_SINTOMAS'])